In [2]:
#Import the libraries

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re

In [4]:
#Get the webpage and parse it as html

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFs1e6_7QdW")
soup = BeautifulSoup(page.content, 'html.parser')
html = list(soup.children)[2]
body = list(html.children)[3]        # Required data is nested here

In [71]:
#Retrieve the day names nested in body 

day_names = body.find_all('div', class_ = 'col-sm-2 forecast-label') #day names are in this tag
list_of_days = []

for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)
#print(list_of_days)

['Overnight', 'Thursday', 'Thursday Night', 'Friday', 'Friday Night', 'Saturday', 'Saturday Night', 'Sunday', 'Sunday Night', 'Monday', 'Monday Night', 'Tuesday', 'Tuesday Night', 'Wednesday']


In [76]:
#Retrieve the weather summary nested inside the body and find temperatures

day_summary = body.find_all('div', class_ = 'col-sm-10 forecast-text') #day summary is in this tag

summary_list = []
for summary in day_summary:
    e = summary.text              
    summary_list.append(e)

summary_string = ' '.join(summary_list)              #convert list to string to retrieve other information
temp = re.findall("[0-9]\d[.]", summary_string)      #regex searches the temp. Eg. '40.' pattern
temp = ' '.join(temp).replace('.','').split()        #remove the period at the end of each number
#print(temp)

In [85]:
#Convert Fahrenheit to celcius 
temp_celcius = []
def celcius(temp):
    for each_temp in temp:
        each_temp = int(each_temp)*(5/9)         #converting to int type as it gives an error for float
        new_temp = round(each_temp)
        temp_celcius.append(f'{new_temp} °C')
    return temp_celcius
#celcius(temp)

In [58]:
#create a dynamic date format

todays_date = date.today()
final_date = todays_date + timedelta(days=6)
delta = todays_date - final_date                     # make a delta for range
date_range = pd.date_range(start=todays_date, end=final_date) 
date_range = date_range.repeat(2)                    #Each day has morning and evening
print(date_range)

DatetimeIndex(['2021-03-25', '2021-03-25', '2021-03-26', '2021-03-26',
               '2021-03-27', '2021-03-27', '2021-03-28', '2021-03-28',
               '2021-03-29', '2021-03-29', '2021-03-30', '2021-03-30',
               '2021-03-31', '2021-03-31'],
              dtype='datetime64[ns]', freq=None)


In [84]:
df = pd.DataFrame({'Date':date_range,'Day':list_of_days, 'Temperature':temp_celcius,
                   'summary':summary_list}) 
print(df)

#df.to_csv('Weather_forecast_easy.csv', index=False)

         Date             Day Temperature  \
0  2021-03-25       Overnight       27 °C   
1  2021-03-25        Thursday       34 °C   
2  2021-03-26  Thursday Night       26 °C   
3  2021-03-26          Friday       38 °C   
4  2021-03-27    Friday Night       26 °C   
5  2021-03-27        Saturday       39 °C   
6  2021-03-28  Saturday Night       27 °C   
7  2021-03-28          Sunday       38 °C   
8  2021-03-29    Sunday Night       27 °C   
9  2021-03-29          Monday       36 °C   
10 2021-03-30    Monday Night       26 °C   
11 2021-03-30         Tuesday       38 °C   
12 2021-03-31   Tuesday Night       27 °C   
13 2021-03-31       Wednesday       40 °C   

                                              summary  
0   Partly cloudy, with a low around 48. West nort...  
1   Sunny, with a high near 62. West wind 6 to 16 ...  
2   Clear, with a low around 46. West wind 9 to 14...  
3   Sunny, with a high near 68. North wind around ...  
4   Clear, with a low around 47. West wind 5